# Chicago Open Data API 

Elegimos el dataset de los crimenes en Chicago desde el año 2001.
Vemos que se puede acceder a los datos viendo los primeros registros de cada variable.
Elegimos los 200.000 ultimos registros del dataset.

In [28]:

'''importar librerias'''
from datetime import date

import numpy as np
import pandas as pd
from sodapy import Socrata

MyAppToken = "YvqGPeBbmzUccvBCj8hAHOF2Q"
'''Cargamos los Dataset complementarios'''
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
OnlineDataset = Socrata("data.cityofchicago.org",
                 MyAppToken,
                 username="ivanna.yanel@gmail.com",
                 password="369874120000.IY")
print("Cliente de API Autenticado...")
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
CrimesDataset = OnlineDataset.get("6zsd-86xi", limit=200000)
print("Cargado Data Set Principal Chicago Crimes...")
IUCRdataset= OnlineDataset.get("miqk-suf4", limit=200000)
print("Cargado Data Set de IUCR...")
BeatsDataset = OnlineDataset.get("n9it-hstw", limit=200000)
print("Cargado Data Set de Beats...")
DistrictDataset = OnlineDataset.get("miqk-suf4", limit=200000)


# Convert to pandas DataFrame results_df = pd.DataFrame.from_records(results)

'''convertir los crimenes en matrices'''
print("Transformando los Dataset Cargados a Pandas...")
CrimesDatasetP = pd.DataFrame.from_records(CrimesDataset)
print("Data Set principal convertido...")
print(CrimesDatasetP.head(n=5))

Cliente de API Autenticado...
Cargado Data Set Principal Chicago Crimes...
Cargado Data Set de IUCR...
Cargado Data Set de Beats...
Transformando los Dataset Cargados a Pandas...
Data Set principal convertido...
   arrest  beat                    block case_number community_area  \
0   False  1023        024XX W OGDEN AVE    JB150383             28   
1   False  0412          016XX E 86TH PL    JA366925             45   
2   False  0215          003XX E 47TH ST    JA522842             38   
3    True  1034  026XX S CALIFORNIA BLVD    JA529032             30   
4    True  1221  007XX N SACRAMENTO BLVD    JA545986             23   

                      date                            description district  \
0  2018-02-13T08:20:00.000                         $500 AND UNDER      010   
1  2001-01-01T11:00:00.000    FINANCIAL IDENTITY THEFT OVER $ 300      004   
2  2017-11-23T15:14:00.000                    AGGRAVATED: HANDGUN      002   
3  2017-11-28T21:43:00.000  VIOLENT OFFENDER: ANN

In [29]:
print("Select distinct")
print(CrimesDatasetP.groupby('primary_type').size())
print(CrimesDatasetP.groupby('fbi_code').size())
print(CrimesDatasetP.groupby('beat').size())
print(CrimesDatasetP.groupby('community_area').size())
print(CrimesDatasetP.groupby('arrest').size())
print(CrimesDatasetP.groupby('year').size())
print(CrimesDatasetP.groupby('location_description').size())
print(CrimesDatasetP.groupby('domestic').size())

Select distinct
primary_type
ARSON                                  381
ASSAULT                              13090
BATTERY                              37179
BURGLARY                              9839
CONCEALED CARRY LICENSE VIOLATION       17
CRIM SEXUAL ASSAULT                    902
CRIMINAL DAMAGE                      22443
CRIMINAL TRESPASS                     5890
DECEPTIVE PRACTICE                    8703
DOMESTIC VIOLENCE                        1
GAMBLING                               219
HOMICIDE                               618
HUMAN TRAFFICKING                        3
INTERFERENCE WITH PUBLIC OFFICER       393
INTIMIDATION                           114
KIDNAPPING                             269
LIQUOR LAW VIOLATION                   513
MOTOR VEHICLE THEFT                  10188
NARCOTICS                            19793
NON-CRIMINAL                            10
NON-CRIMINAL (SUBJECT SPECIFIED)         1
OBSCENITY                               27
OFFENSE INVOLVING CHILDRE

# Conversión de datos en las variable 'arrest' y 'domestic'

Modificamos los valores False por 0 y True por 1 en las variables arrest y domestic.

In [30]:
CrimesDatasetP.loc[CrimesDatasetP.arrest == False, 'arrest'] = '0'
CrimesDatasetP.loc[CrimesDatasetP.arrest == True, 'arrest'] = '1'
CrimesDatasetP.loc[CrimesDatasetP.domestic == False, 'domestic'] = '0'
CrimesDatasetP.loc[CrimesDatasetP.domestic == True, 'domestic'] = '1'
print(CrimesDatasetP.groupby('arrest').size())
print(CrimesDatasetP.groupby('domestic').size())

arrest
0    142868
1     57132
dtype: int64
domestic
0    172571
1     27429
dtype: int64


# Nuevo data frame

Seleccionamos las variables que utilizaremos para nuestro algoritmo de Machine Learning.

In [31]:
finalcrimedataset = CrimesDatasetP[['description','beat','domestic','primary_type','location_description','fbi_code','arrest']]

In [32]:
print (finalcrimedataset.head(n=5))

                             description  beat domestic        primary_type  \
0                         $500 AND UNDER  1023        0               THEFT   
1    FINANCIAL IDENTITY THEFT OVER $ 300  0412        0  DECEPTIVE PRACTICE   
2                    AGGRAVATED: HANDGUN  0215        0             ASSAULT   
3  VIOLENT OFFENDER: ANNUAL REGISTRATION  1034        0       OTHER OFFENSE   
4                         ARMED: HANDGUN  1221        0             ROBBERY   

      location_description fbi_code arrest  
0               RESTAURANT       06      0  
1                RESIDENCE       11      0  
2         DEPARTMENT STORE      04A      0  
3  JAIL / LOCK-UP FACILITY       26      1  
4                 SIDEWALK       03      1  
